In [ ]:
import os
import requests
import google.generativeai as genai
import json

# Configure the Gemini LLM model
genai.configure(api_key=os.environ["API_KEY"])

# Fetch Latest News Articles
def fetch_latest_news(query):
    # Ensure this environment variable is set
    api_key = os.getenv("NEWS_API_KEY")
    url = f"https://newsapi.org/v2/everything?q={query}&apiKey={api_key}&language=en&pageSize=5"
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json().get("articles", [])
        # Filter out articles with missing content or placeholder URLs
        return [article for article in articles 
                if article.get("content") 
                and article.get("url") 
                and "removed.com" not in article["url"]]
    else:
        print("Error fetching news:", response.status_code)
        return []

# Summarize Each Article
def summarize_article(article_content):
    prompt = f"Summarize the following news article in 2 sentences:\n\n{article_content}"
    response = genai.GenerativeModel("gemini-1.5-flash").generate_content(prompt)
    return response.text.strip() if response else "Summary not available"

# Analyze Overall Sentiment
def analyze_overall_sentiment(summaries):
  prompt = f"Analyze the following summaries and provide an overall sentiment (positive, negative, or neutral) in one sentence: do not restate the summaries. Start the sentence with the overall sentiment is \n\n{summaries}"
  response = genai.GenerativeModel("gemini-1.5-flash").generate_content(prompt)
  return response.text.strip()

# Analyze Article Sentiment
def analyze_sentiment(summary):
    prompt = f"Determine the sentiment (positive, negative, or neutral) of the following summary:\n\n{summary}"
    response = genai.GenerativeModel("gemini-1.5-flash").generate_content(prompt)
    return response.text.strip() if response else "Sentiment analysis not available"

# Display Results
def display_results(articles, query):
    
    # Analyze overall sentiment
    summaries = [summarize_article(article['content'] or article['description'] or article['title']) for article in articles]
    overall_sentiment = analyze_overall_sentiment(summaries)
    print(f"\n'{query}': {overall_sentiment}\n")
    print("-" * 50)

    for i, article in enumerate(articles):
        print(f"Article {i+1}: {article['title']}")
        print(f"Source: {article['source']['name']}")
        published_at = article.get("publishedAt")  # Get the published date
        formatted_date = published_at[:10] if published_at else "Date not available"  # Truncate to YYYY-MM-DD
        print(f"Published: {formatted_date}")
        print(f"URL: {article['url']}\n")
        
        # Generate summary
        print("Summary:")
        summary = summarize_article(article['content'] or article['description'] or article['title'])
        print(summary)
        
        # Generate sentiment analysis
        print("\nSentiment:")
        sentiment = analyze_sentiment(summary)
        print(sentiment)
        print("-" * 50)


def main():
    # Get user input
    query = input("Enter a topic to search for news articles: ")
    
    # Fetch articles
    articles = fetch_latest_news(query)
    
    # Display the articles
    if articles:
        display_results(articles, query)
    else:
        print("No articles found for this topic.")


if __name__ == "__main__":
    main()